<a href="https://colab.research.google.com/github/StetskoSergey/pida5_stetsko/blob/master/aml6%20dz%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
from surprise import Dataset, Reader, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import train_test_split, KFold
from surprise import accuracy

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [0]:
users = pd.read_csv('users.dat',sep='::',names = ['UserID','sex','age','Occupation','Zip'])
movies = pd.read_csv('movies.dat',sep='::',names =['MovieID','Title','Genres'])
ratings = pd.read_csv('ratings.dat',sep='::', names = ['UserID','MovieID','Rating','Timestamp'])

In [8]:
users.head()

,UserID,sex,age,Occupation,Zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [0]:
reader = Reader(rating_scale=(1,5))
dataset = Dataset.load_from_df(ratings[['UserID','MovieID','Rating']], reader)

In [0]:
# Функция для оценки точности по датасету, с произвольной функцией и параметрами, с разбивкой на 5 частей кроссвалидации
# доработал функцию, чтобы сначала проверять на простом разбиении, а потом уже кроссвалидацией
def result(use_cross_validation, func, k, sim_options, **kwargs):
  scores = []
  if use_cross_validation:
    kfold = KFold(5, random_state = 1000)
    enumer = kfold.split(dataset)
  else:
    enumer = [train_test_split(dataset, random_state = 1000)]   
  for trainset, testset in tqdm_notebook(enumer):
    algo = func(k=k, sim_options=sim_options, **kwargs)
    algo.fit(trainset)
    predictions = algo.test(testset)
    scores.append(accuracy.rmse(predictions))
  
  return np.mean(scores)  

In [29]:
# не стал пересчитывать, осталось с ошибкой в конце, но и так видно, что точность не удовлетворительная
result(KNNBasic, 40, {'name':'cosine', 'user_based':True}, user_based=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9772
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9740
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9773
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9772
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9777


AttributeError: ignored

In [39]:
result(False, KNNBasic, 40, {'name':'msd', 'user_based':True}, user_based=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9209


0.9209110474689175

In [40]:
result(False, KNNBasic, 40, {'name':'pearson', 'user_based':True}, user_based=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9646


0.9645604887795417

In [41]:
result(False, KNNBasic, 40, {'name':'pearson_baseline', 'user_based':True}, user_based=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9265


0.9264945446407319

In [42]:
result(False, KNNWithZScore, 40, {'name':'msd', 'user_based':True}, user_based=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9294


0.929433421181835

In [43]:
result(False, KNNWithMeans, 40, {'name':'msd', 'user_based':True}, user_based=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9277


0.9276720461033685

In [47]:
result(False, KNNWithMeans, 40, {'name':'msd', 'user_based':False})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8850


0.8849995633299795

In [50]:
result(False, KNNWithMeans, 10, {'name':'msd', 'user_based':False})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9120


0.9119797985349944

In [51]:
result(False, KNNWithMeans, 100, {'name':'msd', 'user_based':False})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8856


0.8855652649950349

In [52]:
result(False, KNNWithMeans, 40, {'name':'cosine', 'user_based':False})

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8927


0.8927354377230493

In [53]:
result(False, KNNWithMeans, 80, {'name':'cosine', 'user_based':False})

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8946


0.8946362865557254

In [54]:
result(False, KNNWithMeans, 20, {'name':'cosine', 'user_based':False})

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9030


0.9030023952269779

In [56]:
result(False, KNNWithMeans, 40, {'name':'pearson', 'user_based':False})

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8886


0.8886418861653658

In [69]:
result(False, KNNWithMeans, 40, {'name':'pearson_baseline', 'user_based':False})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8588


0.8587935955508935

In [58]:
result(False, KNNWithZScore, 40, {'name':'pearson', 'user_based':False})

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.8860


0.8860367246404578

In [59]:
result(False, KNNWithZScore, 40, {'name':'cosine', 'user_based':False})

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.8961


0.8961451392574185

In [60]:
result(False, KNNWithZScore, 40, {'name':'msd', 'user_based':False})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8887


0.888650737447888

In [61]:
result(False, KNNWithZScore, 40, {'name':'pearson_baseline', 'user_based':False})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8608


0.8608349496987385

In [62]:
result(False, KNNBasic, 40, {'name':'pearson', 'user_based':False})

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9919


0.9918752888952799

In [63]:
result(False, KNNBasic, 40, {'name':'cosine', 'user_based':False})

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9985


0.998451437488562

In [64]:
result(False, KNNBasic, 40, {'name':'msd', 'user_based':False})

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9135


0.9134767522462233

In [65]:
result(False, KNNBasic, 40, {'name':'pearson_baseline', 'user_based':False})

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9955


0.9954845957487088

In [70]:
result(False, KNNWithMeans, 35, {'name':'pearson_baseline', 'user_based':False})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8583


0.8583091567455458

In [71]:
result(False, KNNWithMeans, 45, {'name':'pearson_baseline', 'user_based':False})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8593


0.8593104388713794

In [72]:
result(False, KNNWithMeans, 40, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':50})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8610


0.8609766867308739

In [73]:
result(False, KNNWithMeans, 40, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':150})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8581


0.8580704940550926

In [74]:
result(False, KNNWithMeans, 40, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':200})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8577


0.8576995629749785

In [75]:
result(False, KNNWithMeans, 30, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':150})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8571


0.857124553708254

In [76]:
result(False, KNNWithMeans, 25, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':250})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8564


0.8563624888736542

In [79]:
result(False, KNNWithMeans, 20, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':250})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8569


0.8568947010497687

In [80]:
result(False, KNNWithMeans, 15, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':250})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8588


0.8587844417178193

In [77]:
result(False, KNNWithMeans, 20, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':500})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8568


0.856847294680375

In [78]:
result(False, KNNWithMeans, 35, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':500})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8567


0.8567274395915184

In [37]:
result(True, KNNBaseline, 40, {'name':'pearson_baseline', 'user_based':True}, user_based=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8952
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8964
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8935
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8964
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8945
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8960


0.8953218927784601

In [83]:
result(True, KNNWithMeans, 25, {'name':'pearson_baseline', 'user_based':False, 'shrinkage':250})

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8549
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8563
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8574
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8542
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8564


0.8558459453977612